#  Vizualizace a regresní modely

V tomto dokumentu se zaměříme na praktické použití vizualizací a regresních modelů v jazyce **R** s využitím balíčku `ggplot2`. Budeme pracovat se dvěma klasickými datasety `cars` a `mtcars`, na kterých si vyzkoušíme následující:

- tvorbu základních grafů (`geom_point()` a `geom_smooth()`) pro znázornění vztahů mezi proměnnými,
- sestavení jednoduchého regresního modelu pomocí `lm()` a interpretaci jeho výstupu,
- porovnání lineárního a kvadratického modelu na datech `cars`,
- vytvoření a porovnání několika základních modelů na datech `mtcars`.

🎯  Cílem je porozumět základům modelování závislostí a vyzkoušet si hodnocení kvality a vhodnosti zvolených modelů jak vizuálně, tak pomocí výstupů regresní analýzy.

▶️ Načtení knihovny a změna velikosti obrázků.

In [ ]:
# načtení knihovny tidyverse
library(tidyverse)

# změna velikosti obrázků
options(repr.plot.width = 9, repr.plot.height = 6)

## Vizualizace pomocí `ggplot()`

Balíček `ggplot2` umožňuje vytvářet přehledné a flexibilní grafy pomocí tzv. vrstvení jednotlivých komponent. Základem je funkce `ggplot()`, ve které definujeme data a `aes()` pro přiřazení proměnných k vizuálním prvkům (osa x, osa y, barva, velikost aj.). Na tuto základní vrstvu se přidávají další vrstvy, jako např. `geom_point()` pro body nebo `geom_smooth()` pro regresní křivky.

Příklad níže ukazuje jednoduchý bodový graf s lineární regresní přímkou. Graf znázorňuje vztah mezi rychlostí a brzdnou dráhou. 

▶️ Ukázka tvorby grafu

In [ ]:
# základní bodový graf
ggplot(data = cars, aes(x = speed, y = dist)) + # v rámci aes() je definováno, co přijde na osu x a co na osu y
  geom_point()

Pro základní porozumnění datům je to super jednoduché. Navíc takto vytořený graf se dá stejně jednoduše doplňovat o další vrstvy. Například přidáním `geom_smooth()` s metodou `"lm"` doplníme lineární regresní přímku s intervalem spolehlivosti.

▶️ Proložení přímkou

In [ ]:
ggplot(data = cars, aes(x = speed, y = dist)) +
  geom_point() +
  geom_smooth(method = "lm", formula = 'y ~ x')

### 🔍 Kontrola:

Zkontroluj tvar grafu – regresní čára by měla stoupat.

### 🧠 Úkol

Stejným způsobem vytvoříme graf vztahu mezi hmotností a spotřebou z balíku `mtcars`.

1. Na osu x vyneste hmotnost `wt` a na osu y spotřebu `mpg`.
2. Ve funkci `geom_smooth()` použijte metodu lineární regrese `"lm"` v parametru `method`.
3. Ve funkci `geom_smooth()` změňte barvu čáry (`color`) a barvu konfidenčního intervalu (`fill`) na "darkgreen".

In [ ]:
ggplot(data = mtcars, aes(x = ____, y = ____)) +
  geom_point() +
  geom_smooth(method = "____", se = TRUE, color = "____", ____ = "____")

### 🔍 Kontrola:

Zkontroluj, že na vodorovné ose je `wt` a svislé ose `mpg` a že regresní přímka má klesající trend.

## Regresní modely pomocí funkce `lm()`

Stejně jednoduché jako tvorba základních grafů je i statistické vyhodnocení pomocí regresního modelu. Cílem kapitoly není podrobný popis jak a kdy používat regresní modely, ale pouze ukázat, jak jednoduché je to s pomocí **R** a správně připravených dat.

Funkce `lm()` má celou řadu nastavení (můžete vyzkoušet spuštěním `help(lm)`), ale pro naše účely postačí parametry `data` a `formula`. V rámci prvního určíme dataset a v rámci druhého formulujeme vztah, který chceme prověřit. 

Pro vypsání vybraných výstupů z modelu využijeme funkci `summary()`.

In [ ]:
# jednoduchý regresní model
reg_model <- lm(data = cars, formula = dist ~ speed)

summary(reg_model)

## 📊 Výstup lineárního modelu `lm(dist ~ speed)`

Vysvětlení vybraných částí výstupu:

Model má tvar: $  dist = -17.58 + 3.93 * speed$

### 🧪 Statistická významnost

- Oba koeficienty jsou statisticky významné:
  - `Intercept`: p = 0.0123
  - `speed`: p < 0.001 (silná závislost)

---

### 📈 Kvalita modelu

- **R-squared** = 0.6511 → model vysvětluje přibližně 65 % variability v brzdné dráze.
- **Adjusted R-squared** = 0.6438
- **F-statistika** = 89.57, p < 0.001 → model jako celek je silně významný.

---

### 📉 Rezidua

- Minimální reziduum: -29.07
- Maximální reziduum: +43.20
- Medián reziduí: -2.27
- Šířka reziduí naznačuje rozptyl hodnot kolem predikce, ale bez extrémního zkreslení.

---

### ⚠️ Závěr

- Brzdná dráha roste přibližně o **3.93 stop** s každým zvýšením o 1 míli za hodinu.
- Model je **statisticky významný** a **prakticky interpretovatelný**, ale **nelineární vztah** by mohl být ještě přesnější (např. kvadratická složka).  

## Kvadratický model

Je kvadratický model opravdu lepší? Stačí změnit formulaci modelu na `formula = dist ~ poly(speed, 2)` a porovnat výstupy.

▶️

In [ ]:
model_quad <- lm(formula = dist ~ poly(speed, 2), data = cars)
summary(model_quad)

## Porovnání modelů

| Model                  | R²     | Adjusted R² | Residual Std. Error | Významný nelineární člen |
|------------------------|--------|--------------|----------------------|---------------------------|
| Lineární    | 0.6511 | 0.6438       | 15.38                | –                         |
| Kvadratický | 0.6673 | 0.6532       | 15.18                | Ne (p = 0.136)            |

**Lineární model:** jednodušší, dobře interpretovatelný, statisticky silný.

**Kvadratický model:** mírně lepší fit, ale přidaný člen není významný → použití závisí na tom, zda očekáváme nelineární vztah (např. fyzikálně kvadratická závislost).

* Kvadratický model má mírně lepší přizpůsobení datům (vyšší R², nižší chyba).
* Druhý (kvadratický) člen **není statisticky významný** – jeho přidání model výrazně nevylepšuje.
* Pokud **není očekáván nelineární vztah**, je vhodnější **jednodušší lineární model**.

## 🧠 Úkol - vizuální srovnání modelů

1. Jako data použijte dataset `cars` a doplňte chybějící název sloupce na osu y.
2. Doplňte vrstvu s body (`geom_point()`).
3. Doplňte regresní metodu.
4. Doplňte stupeň polynomu a jinou barvu než modrou pro lepší přehlednost.

In [ ]:
# Minimalistická varianta vykreslení
ggplot(data = ____, aes(x = speed, y = ____)) +
   ____() +
   geom_smooth(method = "____", formula = 'y ~ x', se = TRUE) +
   geom_smooth(method = "lm", formula = y ~ poly(x, ____), se = TRUE, col = "____")

## Tvorba grafu

Doteď jsme vytvářely pouze informativní obrázky, které jsou vhodné pro základní přehled o datech, ale nikoli pro jakoukoli formu prezentace nebo publikace. V následující ukázce je představeno přidání několika dalších vrstev, které umožňují přizpůsobení obrázku podle vašich potřeb, aby z něj vznikl skutečný graf.

Možností je nepřeberné množství. V následující ukázce vybíráme možnost změnit popisky os pomocí `labs()`, zmenit vizuální vzhled pomocí `theme` (použito předdefinované `theme_minimal()` a nastavení legendy pomocí `scale_color_manual()`.

▶️ Ukázka

In [ ]:
# Lepší vykreslení
ggplot(cars, aes(x = speed, y = dist)) +
  geom_point(color = "black") +
  geom_smooth(aes(color = "Lineární"), method = "lm", se = FALSE, linetype = "dashed", formula = y ~ x) +
  geom_smooth(aes(color = "Kvadratická"), method = "lm", formula = y ~ poly(x, 2), se = FALSE) +
  labs(
    title = "Srovnání lineárního a kvadratického modelu",
    x = "Rychlost (mph)",
    y = "Délka brzdné dráhy (ft)",
    color = "Regrese"
  ) +
  theme_minimal() +
  scale_color_manual(values = c("Lineární" = "blue", "Kvadratická" = "red"))


## Regrese na balíku `mtcars`

V poslední části si vyzkoušíte vytvořit regresní modely s datasetem `mtcars`. V první fázi zjistíte, jestli je statistická významnost mezi spotřebou a hmotností vozidla. Následně si zkusíte rozšířit model ještě o výkon motoru.

## 🧠 Úkol

1. Do proměnné `lm_model` načtěte výsledek regresního modelu závoslosti spotřeby na hmotnosti vozidla pro dataset `mtcars`.
2. Vypište výsledky modelu.

* Jedná se o dobrý model? Zkontrolujte například `R-squared`. 
* Je model statisticky významný? Napoví `p-value`.

In [ ]:
# model lineární regrese
lm_model <- ____(formula = mpg ~ wt, data = ____)
____(lm_model)

### 🔍 Kontrola

Vztah je záporný (Estimate pro `wt` je -5.34), takže s rostoucí hmotností klesá počet ujetých mil na jednotku množství paliva. Model je statisticky významný, protože `p-value` je menší než 0.05 i 0.01. A podle `R-squared` model vysvětluje 75% variability spotřeby.

## 🧠 Úkol

Ještě vyzkoušíme, jestli dojde ke zlepšení, když do modelu přidáme závislost na  výkonu motoru. Toho se docílí jednoduše změnou parametru `formula`.

Srovnejte `R-squared`. Pokud dojde ke zvýšení, zlepšili jsme model. 

Také můžete v minitabulce `Coefficients:` jednoduše zjistit, jestli je závislost na výkonu statisticky významná. Stačí porovnat, jestli je na příslušném řádku `hp` v posledním sloupci hodnota menší než 0.05. Představu o významnosti dávají i znaky `*`. Čím je jich více, tím lépe.

▶️ Ukázka

In [ ]:
model <- lm(mpg ~ wt + hp, data = mtcars)
summary(model)

## Přidání dalšího rozměru do ggplot grafů

Pokud máme více než dvě proměnné, můžeme v ggplot využít další způsoby vyjádření závislosti například pomocí barvy nebo velikosti bodů. Obě varianty mohou být oddělené, ale v následující ukázce vyjadřují totéž.

▶️ Ukázka

In [ ]:
ggplot(mtcars, aes(x = hp, y = wt, col = mpg, size = mpg)) +
  geom_point()

### 🧠 Úkol - přizpůsobení grafu

V následující ukázce si zkuste libovolně měnit parametry. Například vkládáním jiných sloupců na osy x a y nebo do velikosti bodů či barvy bodů. Můžete měnit popisky nebo akomentovat některé řádky přidáním znaku `#` na začátek řádku a sledovat, co se bude dít.

In [ ]:
# graf pro vaše pokusy
# pokud si chcete jeho základní verzi zachovat, zkopírujte ho do následující buňky a upravujte tam

ggplot(mtcars, aes(x = hp, y = wt, col = mpg, size = cyl)) +
  geom_point(alpha = 0.7) +
  scale_color_viridis_c(option = "plasma") +
#  guides(color = guide_legend(), size = guide_legend()) +  # spojí legendu pokud size a col vychází z jenoho sloupce
  scale_size(range = c(4, 8)) +
  labs(
#    title = "Závislost výkonu a hmotnosti na spotřebě",
    x = "Výkon (hp)",
    y = "Hmotnost (1000 lbs)",
    color = "Spotřeba (mpg)",
    size = "Počet válců (cyl)"
  ) +
  theme_minimal()
  

In [ ]:
# další prostor pro pokusy
